In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    average_precision_score, precision_recall_curve, roc_curve,
    precision_score, recall_score, f1_score, accuracy_score
)
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')

# Load dataset
file_path = '/content/drive/My Drive/dataset.csv'
df = pd.read_csv(file_path)

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())


Dataset loaded successfully!
Shape: (31498, 688)

First 5 rows:
         id target sample_type  continuous_0  categorical_0  categorical_1  \
0  14537510   good         dev           1.0            0.0            0.0   
1  12527950   good         dev           1.0            1.0            1.0   
2  12277478   good         dev           1.0            1.0            1.0   
3  13362203   good         dev           1.0            1.0            2.0   
4  14036057    bad         dev           1.0            1.0            1.0   

   categorical_2  categorical_3  categorical_4  categorical_5  ...  \
0            0.0            0.0            0.0            0.0  ...   
1            1.0            1.0            1.0            0.0  ...   
2            1.0            1.0            2.0            0.0  ...   
3            2.0            2.0            3.0            0.0  ...   
4            1.0            1.0            1.0            0.0  ...   

   continuous_660  continuous_661  continuous_

In [ ]:
# General preliminary view of the dataset
print("Dataset statistics:")
print(df.describe())

# Identify categorical and continuous columns
categorical_cols = [col for col in df.columns if 'categorical' in col.lower()]
continuous_cols = [col for col in df.columns if 'continuous' in col.lower()]
total_cols_count = len(df.columns)

print(f"There are {len(categorical_cols)} categorical columns and {len(continuous_cols)} continuous columns among {total_cols_count} columns")

# Split the dataset based on 'sample_type' column
df_dev = df[df['sample_type'] == 'dev'].copy()
df_test = df[df['sample_type'] == 'test'].copy()
df_valid = df[df['sample_type'] == 'valid'].copy()

print(f"\nDataset split sizes: Dev={len(df_dev)}, Test={len(df_test)}, Valid={len(df_valid)}")

Dataset statistics:
                 id  continuous_0  categorical_0  categorical_1  \
count  3.149800e+04  31498.000000   31498.000000   31498.000000   
mean   1.254142e+07      0.983777       1.352022       8.705188   
std    2.301706e+06      0.126335       1.002153       6.513199   
min    9.004705e+06      0.000000       0.000000       0.000000   
25%    1.035565e+07      1.000000       1.000000       3.000000   
50%    1.230090e+07      1.000000       1.000000       8.000000   
75%    1.447697e+07      1.000000       1.000000      13.000000   
max    1.712175e+07      1.000000       7.000000      26.000000   

       categorical_2  categorical_3  categorical_4  categorical_5  \
count   31498.000000   31498.000000   31498.000000   31498.000000   
mean        8.717823       8.729126       8.936980       0.512890   
std         6.527480       6.529695       6.759147       0.982387   
min         0.000000       0.000000       0.000000       0.000000   
25%         3.000000       3.00

Means there are 'id', 'target', and 'sample_type' columns (3), and all others contain whether 'categorical' (15) or 'continuous' (492) word in their names (3 + 15 + 492 = 510). The split is done before any preprocessing in order to avoid data leakage.

In [ ]:
# Identify columns with more than 30% NaN values in dev set
nan_threshold_perc = 30
nan_percentages_dev = df_dev.isna().mean() * 100
high_nan_cols = nan_percentages_dev[nan_percentages_dev > nan_threshold_perc]

print(f"Number of columns with more than {nan_threshold_perc}% NaN in DEV set: {len(high_nan_cols)}")
if len(high_nan_cols) > 0:
    print(f"Columns to be dropped: {list(high_nan_cols.index[:10])}..." if len(high_nan_cols) > 10 else f"Columns to be dropped: {list(high_nan_cols.index)}")

# Drop high NaN columns from all splits
df_dev.drop(columns=high_nan_cols.index, inplace=True)
df_test.drop(columns=high_nan_cols.index, inplace=True)
df_valid.drop(columns=high_nan_cols.index, inplace=True)

print(f"\nAfter dropping high NaN columns:")
print(f"Dev shape: {df_dev.shape}")
print(f"Test shape: {df_test.shape}")
print(f"Valid shape: {df_valid.shape}")

# Update column lists after dropping
categorical_cols = [col for col in df_dev.columns if 'categorical' in col.lower()]
continuous_cols = [col for col in df_dev.columns if 'continuous' in col.lower()]

print(f"Updated: {len(categorical_cols)} categorical columns and {len(continuous_cols)} continuous columns")

Number of columns with more than 30% NaN in DEV set: 178
Columns to be dropped: ['continuous_471', 'continuous_472', 'continuous_480', 'continuous_481', 'continuous_482', 'continuous_483', 'continuous_484', 'continuous_485', 'continuous_486', 'continuous_487']...

After dropping high NaN columns:
Dev shape: (19368, 510)
Test shape: (6130, 510)
Valid shape: (6000, 510)
Updated: 15 categorical columns and 492 continuous columns


The 178 columns with more than 30% of NaN values were deleted, which is a good practise of preprocessing. The assession was made based only on the dev set, so that now data leakage happens.

In [ ]:
def impute_data(df_subset, categorical_cols, continuous_cols):
    """Function to perform imputation on a given dataframe"""
    dataset_name = df_subset['sample_type'].iloc[0]
    print(f"\nImputing {dataset_name} set...")

    # For categorical columns - use mode
    for col in categorical_cols:
        if not df_subset[col].isna().all():
            mode_value = df_subset[col].mode()
            if len(mode_value) > 0:
                df_subset[col] = df_subset[col].fillna(mode_value[0])

    # For continuous columns - use median
    imputer = SimpleImputer(strategy="median")
    df_subset[continuous_cols] = imputer.fit_transform(df_subset[continuous_cols])

    print(f"NaN count after imputation: {df_subset.isna().sum().sum()}")
    return df_subset

# Apply imputation to each split
df_dev = impute_data(df_dev, categorical_cols, continuous_cols)
df_test = impute_data(df_test, categorical_cols, continuous_cols)
df_valid = impute_data(df_valid, categorical_cols, continuous_cols)

print("\n=== Final Dataset Summary ===")
print(f"Dev: {df_dev.shape}, NaN count: {df_dev.isna().sum().sum()}")
print(f"Test: {df_test.shape}, NaN count: {df_test.isna().sum().sum()}")
print(f"Valid: {df_valid.shape}, NaN count: {df_valid.isna().sum().sum()}")


Imputing dev set...
NaN count after imputation: 0

Imputing test set...
NaN count after imputation: 0

Imputing valid set...
NaN count after imputation: 0

=== Final Dataset Summary ===
Dev: (19368, 510), NaN count: 0
Test: (6130, 510), NaN count: 0
Valid: (6000, 510), NaN count: 0


In [ ]:
# Extract features and targets
feature_cols = [col for col in df_dev.columns if col not in ['sample_type', 'target', 'id']]
X_dev = df_dev[feature_cols]
y_dev = df_dev['target']
X_test = df_test[feature_cols]
y_test = df_test['target']
X_valid = df_valid[feature_cols]
y_valid = df_valid['target']

# Encode target labels
label_encoder = LabelEncoder()
y_dev = label_encoder.fit_transform(y_dev)
y_test = label_encoder.transform(y_test)
y_valid = label_encoder.transform(y_valid)

print("Feature and target shapes:")
print(f"Development set: X_dev={X_dev.shape}, y_dev={y_dev.shape}")
print(f"Test set: X_test={X_test.shape}, y_test={y_test.shape}")
print(f"Validation set: X_valid={X_valid.shape}, y_valid={y_valid.shape}")

# Analyze class imbalance
print("\n=== CLASS IMBALANCE ANALYSIS ===")
dev_class_counts = pd.Series(y_dev).value_counts().sort_index()
class_names = label_encoder.classes_

for i, count in enumerate(dev_class_counts):
    percentage = (count / len(y_dev)) * 100
    print(f"{class_names[i]}: {count:,} ({percentage:.2f}%)")

# Calculate scale_pos_weight for XGBoost
scale_pos_weight = len(y_dev[y_dev == 0]) / len(y_dev[y_dev == 1])
print(f"Scale pos weight (for XGBoost): {scale_pos_weight:.2f}")

Feature and target shapes:
Development set: X_dev=(19368, 507), y_dev=(19368,)
Test set: X_test=(6130, 507), y_test=(6130,)
Validation set: X_valid=(6000, 507), y_valid=(6000,)

=== CLASS IMBALANCE ANALYSIS ===
bad: 1,528 (7.89%)
good: 17,840 (92.11%)
Scale pos weight (for XGBoost): 0.09


In [ ]:
# ⚠️ WARNING: This cell takes 20+ minutes to run!
# Only run this if you want to find new hyperparameters
# Otherwise, skip to the next cell which uses pre-found optimal parameters

PERFORM_HYPERPARAMETER_SEARCH = False  # Set to True to run the search

if PERFORM_HYPERPARAMETER_SEARCH:
    print("🔍 Starting hyperparameter search...")
    print("⏰ This will take approximately 20-30 minutes...")

    # Define parameter grid
    param_dist = {
        'n_estimators': [400, 500, 600],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.005, 0.01, 0.05],
        'subsample': [0.5, 0.6, 0.7],
        'colsample_bytree': [0.6, 0.7, 0.8],
        'gamma': [0, 0.1],
        'reg_alpha': [0, 0.01, 0.1],
        'reg_lambda': [0.1, 1],
        'scale_pos_weight': [scale_pos_weight]
    }

    # Initialize XGBoost and RandomizedSearchCV
    xgb = XGBClassifier(random_state=42, eval_metric='logloss', verbosity=0)
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=20,
        cv=stratified_cv,
        scoring='average_precision',
        n_jobs=-1,
        verbose=2,
        random_state=42
    )

    # Perform search
    start_time = time.time()
    random_search.fit(X_dev, y_dev)
    end_time = time.time()

    print(f"Training completed in {(end_time - start_time) / 60:.2f} minutes")
    print("\nBest hyperparameters found:")
    for param, value in random_search.best_params_.items():
        print(f"  {param}: {value}")
    print(f"Best CV Average Precision: {random_search.best_score_:.4f}")

    # Store the best model
    best_xgb_from_search = random_search.best_estimator_

else:
    print("⏭️ Skipping hyperparameter search - using pre-found optimal parameters")

In [ ]:
# 🚀 Use pre-found optimal hyperparameters
# These were found through previous RandomizedSearchCV runs

OPTIMAL_PARAMS = {
    'subsample': 0.5,
    'scale_pos_weight': scale_pos_weight,  # Use calculated value
    'reg_lambda': 1,
    'reg_alpha': 0.01,
    'n_estimators': 400,
    'max_depth': 5,
    'learning_rate': 0.01,
    'gamma': 0,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'eval_metric': 'logloss',
    'verbosity': 0
}

print("🔧 Training XGBoost with optimal parameters...")
print("Optimal parameters:")
for param, value in OPTIMAL_PARAMS.items():
    if param != 'scale_pos_weight':  # Don't print the long float
        print(f"  {param}: {value}")
print(f"  scale_pos_weight: {OPTIMAL_PARAMS['scale_pos_weight']:.3f}")

# Train the model with optimal parameters
start_time = time.time()
best_xgb = XGBClassifier(**OPTIMAL_PARAMS)
best_xgb.fit(X_dev, y_dev)
end_time = time.time()

print(f"✅ Model trained in {(end_time - start_time):.2f} seconds")

🔧 Training XGBoost with optimal parameters...
Optimal parameters:
  subsample: 0.5
  reg_lambda: 1
  reg_alpha: 0.01
  n_estimators: 400
  max_depth: 5
  learning_rate: 0.01
  gamma: 0
  colsample_bytree: 0.8
  random_state: 42
  eval_metric: logloss
  verbosity: 0
  scale_pos_weight: 0.086
✅ Model trained in 39.89 seconds


In [53]:
# Generate predictions
y_pred_dev = best_xgb.predict(X_dev)
y_pred_valid = best_xgb.predict(X_valid)

# Get prediction probabilities (important for fraud detection)
y_prob_dev = best_xgb.predict_proba(X_dev)[:, 1]  # Probability of fraud (class 1)
y_prob_valid = best_xgb.predict_proba(X_valid)[:, 1]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_basic_metrics(y_true, y_pred, dataset_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"\n=== {dataset_name} SET METRICS ===")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")  # Of predicted frauds, how many were actually fraud?
    print(f"Recall:    {recall:.4f}")     # Of actual frauds, how many did we catch?
    print(f"F1-Score:  {f1:.4f}")        # Harmonic mean of precision and recall

# Evaluate both sets
evaluate_basic_metrics(y_dev, y_pred_dev, "DEVELOPMENT")
evaluate_basic_metrics(y_valid, y_pred_valid, "VALIDATION")


=== DEVELOPMENT SET METRICS ===
Accuracy:  0.8214
Precision: 0.9770
Recall:    0.8254
F1-Score:  0.8949

=== VALIDATION SET METRICS ===
Accuracy:  0.8133
Precision: 0.9464
Recall:    0.8442
F1-Score:  0.8923
